In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
pd.set_option('display.max_columns', 30)
types = {'id': int, 'provincia': str, 'ciudad':str}
training_data = pd.read_csv('../../data/TP2/train.csv', dtype = types, usecols=['id','provincia','ciudad'])
evaluation_data = pd.read_csv('../../data/TP2/test.csv', dtype = types, usecols=['id','provincia','ciudad'])

In [3]:
import html
import re
from unicodedata import normalize

def clean_text(text):
    text = html.unescape(text)
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", 
                    r"\1", 
                    normalize("NFD", text), 
                    0, 
                    re.I) #re.sub(pattern, repl, string, count=0, flags=0);
    
    text = normalize('NFC', text)
    text = re.sub('[^a-zA-ZñÑ]+', ' ', text)
    text = text.lower()
    return text

In [4]:
training_data['ciudad'] = training_data['ciudad'].apply(lambda x: clean_text(str(x)))
training_data['provincia'] = training_data['provincia'].apply(lambda x: clean_text(str(x)))
evaluation_data['ciudad'] = evaluation_data['ciudad'].apply(lambda x: clean_text(str(x)))
evaluation_data['provincia'] = evaluation_data['provincia'].apply(lambda x: clean_text(str(x)))

In [5]:
cambios_prov = {
            'mexico':'edo de mexico',
            'veracruz de ignacio de la llave':'veracruz',
            'coahuila de zaragoza':'coahuila',
            'michoacan de ocampo':'michoacan',
            'baja california':'baja california norte'
          }

def check_prov(provincia):
    if provincia in cambios_prov: provincia = cambios_prov[provincia]
    return provincia

In [6]:
cambios_ciud = {
            'tlaltizapan de zapata':'tlaltizapan de zapata',
            'tlaltizapan':'tlaltizapan de zapata',
            'cancun':'benito juarez',
            'pachuca de soto':'pachuca',
            'medellin de bravo':'medellin',
            'acambay de ruiz castañeda':'acambay',
            'san pedro tlaquepaque':'tlaquepaque',
            'juchitan':'juchitan de zaragoza',
            'general zuazua':'gral zuazua',
            'san pedro mixtepec':'san pedro mixtepec dto ',
            'villahermosa':'tabasco',
            'doctor arroyo':'dr arroyo',
            'yauhquemecan':'tlaxcala',
            'silao de la victoria':'silao',
            'doctor gonzalez':'dr gonzalez',
            'general treviño':'gral treviño',
            'general escobedo':'gral escobedo',
            'general bravo':'gral bravo',
            'playa del carmen':'solidaridad'
          }
def check_ciu(ciudad):
    if ciudad in cambios_ciud: ciudad = cambios_ciud[ciudad]
    return ciudad

In [7]:
training_data['ciudad'] = training_data['ciudad'].apply(lambda x: check_ciu(x))
evaluation_data['ciudad'] = evaluation_data['ciudad'].apply(lambda x: check_ciu(x))

training_data['prov_ciu'] = training_data['provincia'] + '_' + training_data['ciudad']
training_data = training_data[['id','prov_ciu']]

evaluation_data['prov_ciu'] = evaluation_data['provincia'] + '_' + evaluation_data['ciudad']
evaluation_data = evaluation_data[['id','prov_ciu']]

In [8]:
mexico = pd.read_csv('../../data/TP2/Todo_mexico.csv')
mexico.columns = ['provincia', 'ciudad', 'grado_de_marginacion','ambito','grado_rezago_social']
mexico = mexico.dropna()
mexico['ciudad'] = mexico['ciudad'].apply(lambda x: clean_text(str(x)))
mexico['provincia'] = mexico['provincia'].apply(lambda x: clean_text(str(x)))
mexico.provincia = mexico.provincia.apply(lambda x: check_prov(x))
mexico.ciudad = mexico.ciudad.apply(lambda x: check_ciu(x))

In [9]:
grado_rezago_social = mexico.groupby(['provincia','ciudad'])['grado_rezago_social'].value_counts(True).to_frame().unstack()
grado_rezago_social.columns = grado_rezago_social.columns.get_level_values(1).astype(str)
grado_rezago_social['grado_rezago_social'] = grado_rezago_social.T.idxmax()
grado_rezago_social.reset_index(inplace = True)
grado_rezago_social['prov_ciu'] = grado_rezago_social['provincia'] + '_' + grado_rezago_social['ciudad']
grado_rezago_social = grado_rezago_social[['prov_ciu','grado_rezago_social']]
grado_rezago_social

grado_rezago_social,prov_ciu,grado_rezago_social
0,aguascalientes_aguascalientes,Muy bajo
1,aguascalientes_asientos,Muy bajo
2,aguascalientes_calvillo,Muy bajo
3,aguascalientes_cosio,Muy bajo
4,aguascalientes_el llano,Muy bajo
...,...,...
2449,zacatecas_villa garcia,Muy bajo
2450,zacatecas_villa gonzalez ortega,Bajo
2451,zacatecas_villa hidalgo,Bajo
2452,zacatecas_villanueva,Bajo


In [10]:
grado_de_rezago_training = training_data.merge(grado_rezago_social, on='prov_ciu', how='left')
grado_de_rezago_training = grado_de_rezago_training[['id','grado_rezago_social']]
grado_de_rezago_training.fillna(value={'grado_rezago_social':'no definido'}, inplace = True)


grado_de_rezago_evaluation = evaluation_data.merge(grado_rezago_social, on='prov_ciu', how='left')
grado_de_rezago_evaluation = grado_de_rezago_evaluation[['id','grado_rezago_social']]
grado_de_rezago_evaluation.fillna(value={'grado_rezago_social':'no definido'}, inplace = True)

In [11]:
grado_de_rezago_training.shape

(240000, 2)

In [12]:
grado_de_rezago_evaluation.shape

(60000, 2)

In [13]:
training_entities = grado_de_rezago_training[['grado_rezago_social']].to_numpy()
evaluation_entities = grado_de_rezago_evaluation[['grado_rezago_social']].to_numpy()
all_entities = np.unique( np.concatenate( (training_entities, evaluation_entities), axis=None) )

In [14]:
for entity in all_entities:
    grado_de_rezago_training['grado_de_rezago_'+entity] = grado_de_rezago_training['grado_rezago_social'].apply(lambda x: 1 if x == entity else 0)
    grado_de_rezago_evaluation['grado_de_rezago_'+entity] = grado_de_rezago_evaluation['grado_rezago_social'].apply(lambda x: 1 if x == entity else 0)
    
grado_de_rezago_training.drop(columns=['grado_rezago_social'], inplace = True)
grado_de_rezago_evaluation.drop(columns=['grado_rezago_social'], inplace = True)
grado_de_rezago_training.head()

,id,grado_de_rezago_Alto,grado_de_rezago_Bajo,grado_de_rezago_Medio,grado_de_rezago_Muy alto,grado_de_rezago_Muy bajo,grado_de_rezago_no definido
0,254099,0,0,0,0,1,0
1,53461,0,1,0,0,0,0
2,247984,0,0,0,0,1,0
3,209067,0,1,0,0,0,0
4,185997,0,0,0,0,1,0


In [15]:
grado_de_rezago_evaluation.shape

(60000, 7)

In [16]:
grado_de_rezago_training.to_csv('../../res/ftr/encoded_rezago_degree_train.csv')
grado_de_rezago_evaluation.to_csv('../../res/ftr/encoded_rezago_degree_evaluation.csv')